In [77]:
import statistics
import random
import numpy as np
from scipy.optimize import minimize
import scipy.stats as stats
import matplotlib.pyplot as plt
from pymc3.distributions.mixture import NormalMixture

In [57]:
#x1+x2=x3
## Создадим генератор наблюдений
def observation(size=10):
    x1_1=stats.norm(loc=10,scale=3).rvs(size//2)
    x1_2=stats.norm(loc=50,scale=3).rvs((size+1)//2)
    x2_1=stats.norm(loc=40,scale=3).rvs(size//2)
    x2_2=stats.norm(loc=90,scale=3).rvs((size+1)//2)
    x1=np.concatenate((x1_1,x1_2))
    x2=np.concatenate((x2_1,x2_2))
    x3=x1+x2+stats.norm(loc=0,scale=2).rvs(size)
    return np.array([x1,x2,x3]).T
#rvs генерит значения
#T транспонирует матрицу. просто иначе получается массив вида [[x11,x12,..],[x21,x22,..],[x31,x32,...]]

In [58]:
a=observation()
print(a[0])
a

[ 8.50514435 38.95928416 47.61156481]


array([[  8.50514435,  38.95928416,  47.61156481],
       [ 10.25881589,  37.39599727,  45.52326114],
       [  9.62278201,  38.7418324 ,  48.40016218],
       [ 11.00209601,  37.47800384,  46.75837185],
       [ 16.45157806,  30.96374742,  47.61142936],
       [ 44.94655016,  87.68056379, 133.06019791],
       [ 46.12747181,  88.65167011, 135.65805414],
       [ 51.1745709 ,  93.37555654, 143.61270089],
       [ 50.67463981,  91.87987513, 142.9722011 ],
       [ 48.88939178,  89.40635512, 138.83292221]])

In [82]:
conditions = ({'type': 'eq', 'fun' : lambda x: np.array([x[0] + x[1] - x[2]])}) #условие x[0]+x[1]-x[2]=0


def bonds(x,obs):#x- вектор, который оптимизирует scipy
    x1=stats.norm.logpdf(obs[0],loc=x[0],scale=2)
    x2=stats.norm.logpdf(obs[1],loc=x[1],scale=2)
    x3=stats.norm.logpdf(obs[2],loc=x[2],scale=2)
 
    return -(x1+x2+x3)#по сути ввожу функциональную связь на x1,x2,x3

def likelihood(x,truth_x):
    #P(X=(x11,x12,x13..)|theta)=P(X=x11|theta)*...P(X=x1n|theta) 
    x1=NormalMixture.dist(mu=np.array([x[0],x[1]]),
                        sigma=np.array([x[6],x[7]]),w=np.array([0.5,0.5])).logp(truth_x[:,0]).eval()
    x2=NormalMixture.dist(mu=np.array([x[2],x[3]]),
                          sigma=np.array([x[8],x[9]]),w=np.array([0.5,0.5])).logp(truth_x[:,1]).eval()
    x3=NormalMixture.dist(mu=np.array([x[4],x[5]]),
                          sigma=np.array([x[10],x[11]]),w=np.array([0.5,0.5])).logp(truth_x[:,2]).eval()
    print(-(x1.sum()+x2.sum()+x3.sum()),end="\r")
    return -(x1.sum()+x2.sum()+x3.sum())

In [100]:
#делаю налюдения
obs = observation(5)
print(obs[:,0])
#np.var(obs[:,2]) 

[ 1.21294808 14.90467731 52.4842386  51.24589122 49.43263396]


In [101]:
#E-step
#нахожу истинные значения
truth_x = np.array([minimize(bonds, x0=i, args=i, constraints=conditions).x for i in obs])

### Нет смысла уменьшать количество итераций, везде по 2 итерации

In [102]:
for i in obs:
    a=minimize(bonds, x0=i, args=i, constraints=conditions)
    print(a)

     fun: 4.891445208970836
     jac: array([-0.09590626, -0.09590626,  0.09590626])
 message: 'Optimization terminated successfully.'
    nfev: 10
     nit: 2
    njev: 2
  status: 0
 success: True
       x: array([ 0.82932303, 41.64871835, 42.47804138])
     fun: 4.837467412850685
     jac: array([-0.01420248, -0.01420248,  0.01420254])
 message: 'Optimization terminated successfully.'
    nfev: 10
     nit: 2
    njev: 2
  status: 0
 success: True
       x: array([14.84786718, 42.38551997, 57.23338715])
     fun: 4.863107446463495
     jac: array([ 0.06689584,  0.06689584, -0.06689584])
 message: 'Optimization terminated successfully.'
    nfev: 10
     nit: 2
    njev: 2
  status: 0
 success: True
       x: array([ 52.75182188,  92.73740318, 145.48922507])
     fun: 4.849317295389741
     jac: array([-0.04665506, -0.04665506,  0.046655  ])
 message: 'Optimization terminated successfully.'
    nfev: 10
     nit: 2
    njev: 2
  status: 0
 success: True
       x: array([ 51.05927108,

### Оптимально 2-3 итерации (сам он делает 7-9)

In [103]:
 
options = dict(maxiter=6, disp=True)


In [104]:
%%time 
thetas=[13,55,35,95,65,130,5,5,5,5,5,5]
#M-step
thetas= minimize(likelihood, x0=thetas, args=truth_x, options=options)
print(thetas)

         Current function value: 49.788768
         Iterations: 6
         Function evaluations: 168
         Gradient evaluations: 12
      fun: 49.788768237289126
 hess_inv: array([[ 1.1420829 ,  0.21280652, -0.243041  , -0.07900555,  0.26229456,
        -0.3628578 , -0.13326242,  0.37107849,  0.02636243,  0.05897341,
        -0.25061782, -0.11718807],
       [ 0.21280652,  1.2887902 , -0.36228104, -0.12705803,  0.39444427,
        -0.5445186 , -0.20241438,  0.58790229,  0.04499297,  0.06435479,
        -0.39060814, -0.18706507],
       [-0.243041  , -0.36228104,  1.41511763,  0.13644688, -0.44901066,
         0.62107381,  0.22967431, -0.63263311, -0.04350526, -0.10033538,
         0.43474002,  0.20721367],
       [-0.07900555, -0.12705803,  0.13644688,  0.18123418, -0.14514317,
         0.20219175,  0.07278611, -0.16353954, -0.00785849, -0.06711842,
         0.13158887,  0.06184065],
       [ 0.26229456,  0.39444427, -0.44901066, -0.14514317,  1.47419766,
        -0.65800911, -0.245